## Analisis Matricial - _borrador_

A continuacion se desarrolla un ejemplo de un desarrollo a mano.

En este cuaderno se detallan todas las variables y todos los pasos
necesarios para el calculo esperado del programa.

123456789012345678901234567890123456789012345678901234567890123456789

In [1]:
# k = matriz de rigidez local
# a = matriz de compatibilidad geometrica

# K = matriz de rigidez global
# a^t*k*a = K

# F = vector de fuerzas

# ε = [teta_a teta_b delta]
# sigma_p = [Ma Mb F]

# T = matriz de traslacion
# Kt = matriz de rigidez total

# sumatoria de (T^t*K*T)_i = Kt

# Vector de deformaciones y giros = r
# Kt^-1*F = r

# a*r = ε
# k*ε = σ

1. Definir: $k_i$, $S$, $a_i$, $T_i$
    - $k_i$: rigidez local de cada barra.
    - $S$: vector de solicitaciones en los nodos.
    - $a_i$: matriz de compatibilidad geometrica de cada barra.
    - $T_i$: matriz de transformacion de cada barra.


2. Rigidez global: $$ a_i^t k_i a_i = K_i $$
- Rigidez del sistema: $$ \sum T_i^t K_i T_i = K_s $$
- Desplazamientos nodales: $$ K_s^{-1} \vec S = \vec r $$
- Desplazamientos nodales por barra: $$ T_i \vec r = \vec r_i $$
- Deformaciones en coordenadas locales: $$ a_i \vec r_i = \vec \epsilon_i $$
- Esfuerzos en coordenadas locales: $$ k_i \vec \epsilon_i = \vec \sigma_i $$
- Esfuerzos en coordenadas globales: $$ \vec S_i = a_i^t \vec \sigma_i $$

In [2]:
from numpy import array, zeros, round as mround, vstack, reshape
from numpy.linalg import inv, solve
from math import pi as π, sin, cos

In [3]:
#input
i = 90 # grados
b = 30 # [cm]
h = 30 # [cm]
L = 300 # [cm]
ϰ = 0 # kappa 1.2 para secciones rectangulares
fc = 19.5 # [MPa] resistencia especifica a la compresion
𝛾 = 2500 # [kg/m3] densidad del hormigon armado
𝜈 = 0.25; # modulo de poisson

# printable info
A = b*h # [cm²]
E = 𝛾**1.5*0.043*fc**0.5/9.80665 # [kg/mm²]
I = b*h**3/12 # [cm⁴]

# _internals_
G = E/(2*(1+𝜈)) # modulo de cizalle
β = 6*E*I*ϰ/(G*A*L**2);

𝛾 = i*π/180 # [rad]
E = E/0.1**2 # [kg/cm²]
s = round(sin(𝛾), 4)
c = round(cos(𝛾), 4)

#= ANALISIS Matricial =#
k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
k33 = A*E/L

k1 = array([
    [k11, k12, 0],
    [k12, k11, 0],
    [0,   0,   k33]
]) # rigidez local

a1 = array([
    [-s/L, c/L, 1,  s/L, -c/L, 0],
    [-s/L, c/L, 0,  s/L, -c/L, 1],
    [c,   s,   0, -c,   -s,   0]
]) # compatibilidad geometrica

K1 = a1.T@k1@a1

In [4]:
#input
i = 0 # grados
b = 30 # [cm]
h = 40 # [cm]
L = 200 # [cm]
ϰ = 0 # kappa 1.2 para secciones rectangulares
fc = 19.5 # [MPa] resistencia especifica a la compresion
𝛾 = 2500 # [kg/m3] densidad del hormigon armado
𝜈 = 0.25; # modulo de poisson

# printable info
A = b*h # [cm²]
E = 𝛾**1.5*0.043*fc**0.5/9.80665 # [kg/mm²]
I = b*h**3/12 # [cm⁴]

# _internals_
G = E/(2*(1+𝜈)) # modulo de cizalle
β = 6*E*I*ϰ/(G*A*L**2);

𝛾 = i*π/180 # [rad]
E = E/0.01 # [kg/cm²]
s = round(sin(𝛾), 4)
c = round(cos(𝛾), 4)

#= ANALISIS Matricial =#
k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
k33 = A*E/L

k3  = array([
    [k11, k12, 0],
    [k12, k11, 0],
    [0,   0,   k33]
]) # rigidez local

a3 = array([
    [-s/L, c/L, 1,  s/L, -c/L, 0],
    [-s/L, c/L, 0,  s/L, -c/L, 1],
    [c,    s,   0, -c,   -s,   0]
]) # compatibilidad geometrica

K3 = a3.T@k3@a3

In [5]:
F = array([10000, -10000, 0, 10000, -10000, 0]).T

T1 = array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0]
])

T2 = array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1]
])

T3 = array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1]
])

Kt = T1.T@K1@T1 + T2.T@K1@T2 + T3.T@K3@T3
r = inv(Kt)@F

In [6]:
r1 = T1@r
r2 = T2@r
r3 = T3@r

ϵ1 = a1@r1
ϵ2 = a1@r2
ϵ3 = a3@r3

σ1 = k1@ϵ1
σ2 = k1@ϵ2
σ3 = k3@ϵ3

σ = array([
    [round(σ1[0]/100), round(σ2[0]/100), round(σ3[0]/100)],
    [round(σ1[1]/100), round(σ2[1]/100), round(σ3[1]/100)],
    [round(σ1[2]), round(σ2[2]), round(σ3[2])]
])

V = array([(σ[0,0] + σ[1,0])/3, (σ[0,1] + σ[1,1])/3, (σ[0,2] + σ[1,2])/2])

print("Ma: ", σ[0,:])
print("Mb: ", σ[1,:])
print("F:  ", σ[2,:])
print("V:  ", V)

Ma:  [ 15774  15774 -14226]
Mb:  [ 14226  14226 -14226]
F:   [-4226 24226     0]
V:   [ 10000.  10000. -14226.]
